In [0]:
import dlt
from pyspark.sql.functions import col

# Configurações de conexão com o PostgreSQL
jdbc_url = "jdbc:postgresql://dpg-cuciu352ng1s73b52bgg-a.ohio-postgres.render.com:5432/transactions_yz0m"
jdbc_properties = {
    "user": "transactions_yz0m_user",
    "password": "66BsNmXaNh3btK1ZY8GLbnrFmwUFK9xY",
    "driver": "org.postgresql.Driver"
}

# Nome da tabela no PostgreSQL
postgres_table = "transactions"

# Função para carregar dados do PostgreSQL
@dlt.table(
    comment="Carrega dados de transações do PostgreSQL"
)
def raw_transactions():
    return (
        spark.read.format("jdbc")
        .option("url", jdbc_url)
        .option("dbtable", postgres_table)
        .option("user", jdbc_properties["user"])
        .option("password", jdbc_properties["password"])
        .option("driver", jdbc_properties["driver"])
        .load()
    )

# Processar apenas os dados novos com base em `transaction_time`
@dlt.table(
    comment="Tabela Delta gerenciada com dados incrementais de transações"
)
def transactions():
    raw_data = dlt.read("raw_transactions")
    # Transformação opcional e deduplicação de dados
    return raw_data.filter(
        col("transaction_time") > dlt.read_stream("transactions").selectExpr("MAX(transaction_date)").collect()[0][0]
    )
